In [89]:
import numpy as np
import pandas as pd
import pickle
import mysql.connector
# data = pd.read_csv("input_datasets/modif_csv_cleaned.csv")

# data = pd.read_csv("input_datasets/next_movies.csv")

# display(data)


## connexion a la DB

In [90]:
# Informations de connexion à la base de données
host = "casq.mysql.database.azure.com"
user = "tenshi"
password = "Simplon59"
database = "db_movies"

# Connexion à la base de données
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Requête SQL pour extraire les données
sql_query = "SELECT * FROM predict_films;"
# sql_query = "SELECT f.*, s.nom AS 'studio_name', p_realisateur.nom AS 'realisateur_nom', p_compositeur.nom AS 'compositeur_nom', p_producteur.nom AS 'producteur_nom', p_acteur.nom AS 'acteur_nom' FROM films f JOIN film_studio fs ON f.film_id = fs.film_id JOIN studios s ON s.studio_id = fs.studio_id JOIN film_personne fp_realisateur ON fp_realisateur.film_id = f.film_id AND fp_realisateur.role = 'realisateur' JOIN personnes p_realisateur ON p_realisateur.personne_id = fp_realisateur.personne_id JOIN film_personne fp_compositeur ON fp_compositeur.film_id = f.film_id AND fp_compositeur.role = 'compositeur' JOIN personnes p_compositeur ON p_compositeur.personne_id = fp_compositeur.personne_id JOIN film_personne fp_producteur ON fp_producteur.film_id = f.film_id AND fp_producteur.role = 'producteur' JOIN personnes p_producteur ON p_producteur.personne_id = fp_producteur.personne_id JOIN film_personne fp_acteur ON fp_acteur.film_id = f.film_id AND fp_acteur.role = 'acteur' JOIN personnes p_acteur ON p_acteur.personne_id = fp_acteur.personne_id;"
# Lecture des données dans un DataFrame pandas
data = pd.read_sql(sql_query, conn)

# Fermeture de la connexion à la base de données
conn.close()

# Enregistrement des données dans un fichier CSV
data.to_csv("datasets/donnees_SQL.csv", index=False)


/var/folders/2d/rl2wdlyn7t93k68_f2b6tmzw0000gn/T/ipykernel_5254/780300566.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_query, conn)


## chargement du modele

In [91]:
with open("models/model_cb.pkl", "rb") as f:
    model = pickle.load(f)
f.close()

## netoyage de la donnée

In [92]:
data['estimation'].fillna(-1, inplace=True)

types_de_donnees = {
    # 'titre': 'object',
    # 'budget': 'float64',
    # 'compositeur': 'object',
    # 'semaine_fr': 'object',
    # 'semaine_usa': 'object',
    # 'duree': 'int64',
    # 'entrees_fr': 'int64',
    # 'franchise': 'int64',
    # 'genres': 'object',
    # 'pegi_fr': 'object',
    # 'pegi_usa': 'object',
    # 'entrees_usa': 'float64',
    # 'salles_fr': 'float64',
    # 'studio': 'object',
    'estimation' : 'int64'
}
data = data.astype(types_de_donnees)

cols_drop = ["acteurs", "producteur", "realisateur","synopsis", "images"]
data = data.drop(cols_drop, axis=1)


display(data)
display(data.info())

/var/folders/2d/rl2wdlyn7t93k68_f2b6tmzw0000gn/T/ipykernel_5254/2553314711.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['estimation'].fillna(-1, inplace=True)


,id,titre,budget,compositeur,semaine_fr,semaine_usa,duree,franchise,genres,pays,pegi_fr,pegi_usa,entrees_usa,salles_fr,studio,estimation,entrees_fr
0,1205,civil war,None,None,2024-04-17,2024-04-12,109.0,None,thriller,u.s.a.,interdit - 12 ans avec avertissement,None,25712608.0,720.0,metropolitan filmexport,474877,None
1,1206,orso,None,None,2024-04-17,None,86.0,None,drame,france,interdit - 12 ans,None,NaN,NaN,les mûres sauvages,131671,None
2,1207,monkey man,None,None,2024-04-17,2024-04-05,120.0,None,thriller,u.s.a.,interdit - 12 ans avec avertissement,None,10119435.0,283.0,universal pictures international france,100012,None
3,1208,spy x family code: white,None,None,2024-04-17,None,110.0,None,thriller,japon,tout public,None,NaN,355.0,sony pictures releasing france,78950,None
4,1209,l'homme aux mille visages,None,None,2024-04-17,None,90.0,None,documentaire,france,tout public,None,NaN,83.0,pyramide distribution,46752,None
5,1210,zaman dark,None,None,2024-04-17,None,93.0,None,thriller,liban,interdit - 16 ans,None,NaN,1.0,visiosfeir,112710,None
6,1211,ici et là-bas,None,None,2024-04-17,None,90.0,None,comédie,france,tout public,None,NaN,638.0,studiocanal,371302,None
7,1212,la rondine (metropolitan opera),None,None,2024-04-20,None,167.0,None,opéra,u.s.a.,tout public,None,NaN,77.0,pathé live,15513,None
8,1213,laroy,None,None,2024-04-17,None,112.0,None,thriller,u.s.a.,tout public,None,NaN,168.0,arp sélection,37452,None
9,1214,jouj,None,None,2024-04-17,None,90.0,None,comédie,maroc,None,None,NaN,42.0,golden afrique ciné,67274,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           22 non-null     int64  
 1   titre        22 non-null     object 
 2   budget       0 non-null      object 
 3   compositeur  0 non-null      object 
 4   semaine_fr   22 non-null     object 
 5   semaine_usa  2 non-null      object 
 6   duree        21 non-null     float64
 7   franchise    0 non-null      object 
 8   genres       22 non-null     object 
 9   pays         22 non-null     object 
 10  pegi_fr      15 non-null     object 
 11  pegi_usa     0 non-null      object 
 12  entrees_usa  2 non-null      float64
 13  salles_fr    21 non-null     float64
 14  studio       22 non-null     object 
 15  estimation   22 non-null     int64  
 16  entrees_fr   0 non-null      object 
dtypes: float64(3), int64(2), object(12)
memory usage: 3.0+ KB


None

In [93]:
data['budget'] = data['budget'].astype(str).str.replace('.0', '')


data['is_compositeur'] = data['compositeur'].apply(lambda x: 1 if x is not None else 0)

data['annee'] = data['semaine_fr'].astype(str).str.slice(0, 4).astype(int)

data["franchise"] = data["franchise"].astype(str)

data = data.loc[data['genres'] != -1]


data["origine"] = data["pays"]
autres_pays = data[(data["pays"] != "etats-unis") & (data["pays"] != "france")]
data.loc[autres_pays.index, "origine"] = "Autre"
cols_drop = ["pays"]
data = data.drop(cols_drop, axis=1)

data.replace('None', None, inplace=True)


# data['salles_fr'] = data['salles_fr'].astype(str).str.replace('.0', '')
# data['salles_fr'] = data['salles_fr'].str.replace(' ', '')
# data['salles_fr'].replace('-', np.nan, inplace=True)
# # Convertir la colonne en float
# data['salles_fr'] = data['salles_fr'].astype(float)
# # Remplacer les NaN par -1
# data['salles_fr'].fillna(-1, inplace=True)
# # Convertir en entie
# data['salles_fr'] = data['salles_fr'].astype(int)
# data = data[data['salles_fr'] != -1]
# # Réinitialiser l'index si nécessaire
# data.reset_index(drop=True, inplace=True)




## prediction des estimation

In [94]:
# Connexion à la base de données
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Créer un curseur pour exécuter les requêtes SQL
cursor = conn.cursor()

# Itérer sur chaque ligne du DataFrame
for index, row in data.iterrows():
    # Créer un DataFrame pour la prédiction
    inputs = pd.DataFrame({
        'budget': [row['budget']],
        'duree': [row['duree']],
        'franchise': [row['franchise']],
        'genres': [row['genres']],
        'pegi_fr': [row['pegi_fr']],
        'pegi_usa': [row['pegi_usa']],
        'entrees_usa': [row['entrees_usa']],
        'salles_fr': [row['salles_fr']],
        'studio': [row['studio']],
        'is_compositeur': [row['is_compositeur']],
        'annee': [row['annee']],
        'origine': [row['origine']],
    })

    # Prédire le nombre d'entrées pour cette ligne
    predicted_entries = model.predict(inputs)

    # Ajouter la prédiction à la liste des prédictions
    prediction = str(predicted_entries[0])  # Convertir en int standard Python

    # Mettre à jour la base de données avec la prédiction
    film_id = row['id']  # Supposons que 'id' soit la clé primaire de la table
    cursor.execute("UPDATE predict_films SET estimation = %s WHERE id = %s", (prediction, film_id))

# Valider la transaction et fermer le curseur
conn.commit()
cursor.close()

# Fermer la connexion à la base de données
conn.close()


/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) franchise, pegi_fr, pegi_usa, studio, is_compositeur.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) franchise, pegi_fr, pegi_usa, studio, is_compositeur.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) franchise, pegi_fr, pegi_usa, studio, is_compositeur.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarnin